In [3]:
from pytket.extensions.nexus import Nexus, QuantinuumConfig
#from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.nexus.backends import NexusBackend
from pytket.backends.resulthandle import ResultHandle
from pytket import Circuit
from pytket.unit_id import BitRegister
from pytket.circuit.display import render_circuit_jupyter
import networkx as nx
import matplotlib.pyplot as plt
import math as ma 
import numpy as np
import random as rm
import pandas as pd
import re
import copy
import warnings
import sympy as sy
import itertools
import ast

import sys
sys.path.append('/home/jovyan/BenchmarkingQuantinuumH1/src')

import helper_functions as hf
import circuits_version_02 as cv2

In [4]:
#my_new_quantinuum_exp = Nexus().get_project_by_name("UBQC")
#configuration = QuantinuumConfig(device_name="H1-1E", user_group="Default")
#backend = QuantinuumBackend(configuration, my_new_quantinuum_exp)

my_new_quantinuum_exp = Nexus().get_project_by_name("UBQC")
configuration = QuantinuumConfig(device_name="H1-1", user_group="Default")
backend = NexusBackend(configuration, my_new_quantinuum_exp)

#from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline
#api_offline = QuantinuumAPIOffline()
#backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)


Started using project with name: UBQC


In [ ]:
circuit = cv2.circuit_01()
compiled_circuit = backend.get_compiled_circuit(circuit)
backend.cost(compiled_circuit,n_shots = 10)